 ### Клонирование репозитория YOLOv7 и установка предварительно обученной модели (от разработчика YOLOv7)


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
!wget 'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e_training.pt' -O yolov7-e6e_training.pt --fuzzy

# Шаг 1 - предварительное обучение на расширенном наборе (Okutama-Action)
### Установка готовой версии набора (5.5 ГБ)

In [ ]:
!wget 'https://downloader.disk.yandex.ru/disk/f24ff239f6466c56ff13f4d667822144f209a2d0465e11610c0a425b4aa16a2d/636d82ed/_ah9MjiPXrIm4fxxD9Ws4J34RI1icUbSBi5hNDyDvDC-OXjCSyxs3RhSKfg_M6VUsEpY79dMn0c5wNkdGe3j-w%3D%3D?uid=0&filename=step_1.zip&disposition=attachment&hash=sl9zGDVuNJK8fNXN9oTS3PPj%2B1w7SStA9v9dHLPzImV9taVwgr1Lf/fNDOujIT1bq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=674100834&fsize=5865785471&hid=6e9000707fa0dd24ef46e343b8f176b1&media_type=compressed&tknv=v2' -O step_1 --fuzzy
!unzip step_1

Перед запуском следующей ячейки измените количество классов в конфигурационном файле yolov7/cfg/training/yolov7-e6e.yaml с 80 на 1

In [ ]:
!python yolov7/train_aux.py --workers 8 --device 0 --batch-size 4 --data step_1/dataset.yaml --img 1280 1280 --epochs 300 --cfg yolov7/cfg/training/yolov7-e6e.yaml --weights 'yolov7-e6e_training.pt' --name yolov7-e6e-custom

Я остановил обучение на пятой эпохе. Одна эпоха на RTX 3090 длится примерно 4 часа

# Шаг 2 - дообучение на предоставленном наборе
### Установка готовой версии набора (0.53 ГБ)

In [ ]:
!wget 'https://downloader.disk.yandex.ru/disk/a25881993dd98d597305aff26df876dfc3da198da523c88a3c4af3238eea08ad/636d8301/_ah9MjiPXrIm4fxxD9Ws4McmYgRq9RIocNHApMIwnR5mRPOlqylXvriElo1rqnuHPTFZHvk7sxZT2I5gG2k9EA%3D%3D?uid=0&filename=step_2.zip&disposition=attachment&hash=oStemeDSXDiD0iPEsfYMkPMwv5jwVBE1q3yvkrKTci2Ctr7kZioVLuw2rvD/Q87/q/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=674100834&fsize=557209045&hid=ecad0d2ae78d5bbe7ba6bbfd79694654&media_type=compressed&tknv=v2' -O step_2 --fuzzy
!unzip step_2

In [ ]:
!python yolov7/train_aux.py --workers 8 --device 0 --batch-size 4 --data step_2/dataset.yaml --img 1280 1280 --epochs 300 --cfg yolov7/cfg/training/yolov7-e6e.yaml --weights 'runs/train/yolov7-e6e-custom/weights/best.pt' --name yolov7

# Шаг 3 - получение предсказаний
### Установите тестовый набор (если его нет)

In [ ]:
!wget 'https://downloader.disk.yandex.ru/disk/d9b0e72388067c4d4842c4ae059001980be5b764592470073902ed3acdae2b66/636d81aa/_ah9MjiPXrIm4fxxD9Ws4JDMdRDqk4uoxSejVILlTZf8z0h0me1BLOx433njnIy2WAFpopifUSgkkMOh0N2rkw%3D%3D?uid=0&filename=test.zip&disposition=attachment&hash=V2nHb8SSCIl172zdlXnQCpQVSm6r54qoSkGrf%2BCVcgTvL99rj84tk9FTiES0AMVHq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=674100834&fsize=2173743880&hid=291a12ba9ea5ae9f8c997fcb2eae7269&media_type=compressed&tknv=v2' -O test --fuzzy
!unzip test

Скачивание моей модели

In [ ]:
!wget https://downloader.disk.yandex.ru/disk/b90209decad6b1382ea46f2642e84d363908abfce1bc2da5cc95cf5d2112c7c5/636d821e/_ah9MjiPXrIm4fxxD9Ws4LDKDpPDig1k38bQ2Xn-jvcaJ7PuHSlCTTZPezONuBb3uqnFMLk_9tLnF3cguAYQdg%3D%3D?uid=0&filename=best.pt&disposition=attachment&hash=4GDpxAuoKJs1WD09BXFHdY7y2N5ZI/4t0vqJHdj3MF6Le3cN8yS60zR37/5/lkoHq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=674100834&fsize=330955063&hid=0dcc08e758be22ef1952405768bdee81&media_type=compressed&tknv=v2 -O best.pt --fuzzy

Если хотите проверить свою модель, измените значение параметра weights на 'runs/train/yolov7/weights/best.pt'

In [ ]:
!python yolov7/detect.py --img-size 1280 --weights best.pt --source test --save-txt --save-conf --conf-thres 0.25 --iou-thres 0.45

# Шаг 4 - подготовка решения
Лучший результат на паблике - 0.647308

In [1]:
import os
import shutil
import pandas as pd
import cv2
import random
import numpy as np
from tqdm import tqdm
import json


def get_soliton_labels_df_simple_solution(path_to_img_folder):
    simple_solution = []
    names = os.listdir(path_to_img_folder)[:-1]
    names2 = sorted([int(i.split('.')[0]) for i in names])
    names = [str(i)+'.JPG' for i in names2]
    img_names = [name for name in os.listdir(r'F:\omsk\test\test') if name[-1]!='e']
    for i in range(len(names)):
        if not names[i] in img_names:
            names[i] = names[i].lower()
    for detection_file in tqdm(names[:]):
        image = cv2.imread(path_to_img_folder + detection_file)
        size = image.shape[:2]
        txt_file = detection_file.split('.')[0] + '.txt'
        if os.path.isfile(path_to_img_folder + 'labels/' + txt_file):
            with open(path_to_img_folder + 'labels/' + txt_file, 'r') as f:
                data = f.read()
                data = [i for i in data.split('\n') if i != '']
        else:
            data = []
        dicts = []
        data.reverse()
        confids = []
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = int(val[0]), val[1:5], val[5]
            confids.append(conf)
            center_x, center_y, width, height = xywh
            radius = min([width, height]) * 1.05

            cx = int(center_x * size[1])
            cy = int(center_y * size[0])
            r  = int(radius * size[1])
            region_shape = f'{{"cx":{cx},"cy":{cy},"r":{r}}}'
            
            if conf > 0.3:
                dicts.append(region_shape)

        if len(confids) > 0:
            if max(confids) < 0.6:
                dicts = []
                data = []

        if dicts == []:
            count_region = 0
        else:
            count_region = len(data)
        if count_region > 0 and dicts != []:
            simple_solution.append([detection_file, dicts])
        else:
            simple_solution.append([detection_file, 0])
    return simple_solution

In [2]:
simple_solution = get_soliton_labels_df_simple_solution('runs/detect/exp/')
simple_solution = pd.DataFrame(simple_solution, columns=['ID_img', 'region_shape'])

100%|██████████████████████████████████████████████████████████████████████████████| 3435/3435 [00:46<00:00, 73.26it/s]


In [3]:
simple_solution.to_csv("predictions.csv", index=False)